# `DataFrame` objects
A DataFrame object represents a spreadsheet, with cell values, column names and row index labels. You can define expressions to compute columns based on other columns, create pivot-tables, group rows, draw graphs, etc. You can see `DataFrame`s as dictionaries of `Series`.

## Creating a `DataFrame`
You can create a DataFrame by passing a dictionary of `Series` objects:

In [1]:
import pandas as pd
import numpy as np

In [2]:
people_dict = {
    "weight": pd.Series([68, 83, 112], index=["alice", "bob", "charles"]),
    "birthyear": pd.Series([1984, 1985, 1992], index=["bob", "alice", "charles"], name="year"),
    "children": pd.Series([0, 3], index=["charles", "bob"]),
    "hobby": pd.Series(["Biking", "Dancing"], index=["alice", "bob"]),
}
people = pd.DataFrame(people_dict)
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


A few things to note:
* the `Series` were automatically aligned based on their index,
* missing values are represented as `NaN`,
* `Series` names are ignored (the name `"year"` was dropped),


![image.png](https://files.realpython.com/media/fig-01.923689cf9afe.png)

In [7]:
people.columns

Index(['weight', 'birthyear', 'children', 'hobby'], dtype='object')

In [8]:
people.index

Index(['alice', 'bob', 'charles'], dtype='object')

In [9]:
people.to_numpy()

array([[68, 1985, nan, 'Biking'],
       [83, 1984, 3.0, 'Dancing'],
       [112, 1992, 0.0, nan]], dtype=object)

If you pass a list of columns and/or index row labels to the `DataFrame` constructor, it will guarantee that these columns and/or rows will exist, in that order, and no other column/row will exist. For example:

In [10]:
d2 = pd.DataFrame(
        people_dict,
        columns=["birthyear", "weight", "height"],
        index=["bob", "alice", "eugene"]
     )
d2

,birthyear,weight,height
bob,1984.0,83.0,NaN
alice,1985.0,68.0,NaN
eugene,NaN,NaN,NaN


Another convenient way to create a `DataFrame` is to pass all the values to the constructor as an `ndarray`, or a list of lists, and specify the column names and row index labels separately:

In [11]:
values = [
            [1985, np.nan, "Biking",   68],
            [1984, 3,      "Dancing",  83],
            [1992, 0,      np.nan,    112]
         ]
d3 = pd.DataFrame(
        data= values,
        columns=["birthyear", "children", "hobby", "weight"],
        index=["alice", "bob", "charles"]
     )
d3

,birthyear,children,hobby,weight
alice,1985,NaN,Biking,68
bob,1984,3.0,Dancing,83
charles,1992,0.0,NaN,112


## Accessing columns
You can access columns pretty much as you would expect. They are returned as `Series` objects:
- see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [12]:
people["birthyear"]

alice      1985
bob        1984
charles    1992
Name: birthyear, dtype: int64

In [13]:
people.birthyear

alice      1985
bob        1984
charles    1992
Name: birthyear, dtype: int64

In [14]:
people[["birthyear"]]

,birthyear
alice,1985
bob,1984
charles,1992


You can also get multiple columns at once:

In [15]:
people[["birthyear", "hobby"]]

,birthyear,hobby
alice,1985,Biking
bob,1984,Dancing
charles,1992,NaN


## Accessing rows by index names

In [16]:
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


The `loc` attribute lets you access rows instead of columns. The result is a `Series` object in which the `DataFrame`'s column names are mapped to row index labels:

In [17]:
people.loc["charles"]

weight        112
birthyear    1992
children        0
hobby         NaN
Name: charles, dtype: object

Selecting subset of rows and columns 

In [18]:
people.loc["charles", "weight"]

112

In [19]:
people.loc[["charles","bob"]]

,weight,birthyear,children,hobby
charles,112,1992,0.0,NaN
bob,83,1984,3.0,Dancing


We can also define which lines and columns to be selected

In [20]:
people.loc[["charles","bob"],["hobby","weight"]]

,hobby,weight
charles,NaN,112
bob,Dancing,83


Finally, you can pass a boolean array to get the matching rows:

In [21]:
people.loc[np.array([True, False, True])]

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
charles,112,1992,0.0,NaN


## Accessing rows by index location
You can also access rows by integer location using the `iloc` attribute:

In [22]:
people.iloc[2]

weight        112
birthyear    1992
children        0
hobby         NaN
Name: charles, dtype: object

You can also get a slice of rows, and this returns a `DataFrame` object:

In [23]:
people.iloc[1:3]

,weight,birthyear,children,hobby
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


Finally, you can pass a boolean array to get the matching rows:

In [24]:
people.iloc[np.array([True, False, True])]

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
charles,112,1992,0.0,NaN


## Conditional selection
An important feature of pandas is conditional selection using bracket notation, very similar to numpy:

#### boolean expression for cell selection of numerical attribute

In [36]:
# Choose only numerical columns (a copy)
people_cat = people[['weight','birthyear','children']]
people_cat > 0

,weight,birthyear,children
alice,True,True,False
bob,True,True,True
charles,True,True,False


In [37]:
people_cat[people_cat > 0]

,weight,birthyear,children
alice,68,1985,NaN
bob,83,1984,3.0
charles,112,1992,NaN


In [38]:
people_cat[people_cat > 0][['children']]

,children
alice,NaN
bob,3.0
charles,NaN


#### boolean expression for row selection

In [39]:
people_cat["birthyear"] < 1990

alice       True
bob         True
charles    False
Name: birthyear, dtype: bool

In [40]:
people_cat[people_cat["birthyear"] < 1990] ="DNA" # warrning, not the best way use loc or iloc

<ipython-input-40-27503fb93d58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_cat[people_cat["birthyear"] < 1990] ="DNA" # warrning, not the best way use loc or iloc
C:\ProgramData\Anaconda3\envs\firstDNA\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [41]:
people_cat

,weight,birthyear,children
alice,DNA,DNA,DNA
bob,DNA,DNA,DNA
charles,112,1992,0


In [82]:
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


we can combine conditions

In [66]:
people[(people['weight']>80) & ~(people['hobby'].isna())]

,weight,birthyear,children,hobby
bob,83,1984,3.0,Dancing


## Index management
Let's discuss some more features of indexing, including resetting the index or setting it 

In [43]:
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


In [44]:
# Reset to default 0,1...n index
people.reset_index()

,index,weight,birthyear,children,hobby
0,alice,68,1985,NaN,Biking
1,bob,83,1984,3.0,Dancing
2,charles,112,1992,0.0,NaN


if the index is no more needed

In [45]:
people.reset_index(drop=True)

,weight,birthyear,children,hobby
0,68,1985,NaN,Biking
1,83,1984,3.0,Dancing
2,112,1992,0.0,NaN


we can add a new index

In [3]:
people['new_inde'] = pd.date_range('1/1/2000', periods=3)
people

,weight,birthyear,children,hobby,new_inde
alice,68,1985,NaN,Biking,2000-01-01
bob,83,1984,3.0,Dancing,2000-01-02
charles,112,1992,0.0,NaN,2000-01-03


In [4]:
# I made mistake on the name of the attribute
people.drop('new_inde',axis=1 ,inplace=True)

In [7]:
people

,weight,birthyear,children,hobby
alice,68,1985,NaN,Biking
bob,83,1984,3.0,Dancing
charles,112,1992,0.0,NaN


In [8]:
people['new_index'] = pd.date_range('1/1/2000', periods=3)
people

,weight,birthyear,children,hobby,new_index
alice,68,1985,NaN,Biking,2000-01-01
bob,83,1984,3.0,Dancing,2000-01-02
charles,112,1992,0.0,NaN,2000-01-03


In [9]:
people.reset_index(drop=True).set_index('new_index')

,weight,birthyear,children,hobby
new_index,,,,
2000-01-01,68,1985,NaN,Biking
2000-01-02,83,1984,3.0,Dancing
2000-01-03,112,1992,0.0,NaN


In [10]:
newind = 'CA NY WY'.split()
people['state'] = newind
people.set_index('state', inplace=True)
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,68,1985,NaN,Biking,2000-01-01
NY,83,1984,3.0,Dancing,2000-01-02
WY,112,1992,0.0,NaN,2000-01-03


## View DataFrame

As we already see with Numpy, when one takes a slice of an array, what one gets is not an entirely new array, but rather a view of the original array. Views share the underlying data of the array from which they were spawned, meaning changes to one impact the other. pandas also often exhibits this behavior, but in some much more nuanced and often deeply problematic ways.

Subsetting Series or DataFrames in pandas will also sometimes generate views, but will also sometimes not. This differs from how views work in numpy: in numpy, the rules for when you get views and when you don’t are a little complicated, but they are consistent: certain behaviors (like a basic slice) will always return a view, and others (fancy slicing) will never return a view.

But in pandas, whether you get a view or not depends on the structure of the DataFrame and, if you are trying to modify a slice, the nature of the modification. 

- https://realpython.com/pandas-settingwithcopywarning/
- https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html (very good)

General rules (there is some exceptions)
- With simple slicing, we get views
- With index, mask, list, ... we get copies


In [11]:
# here is copy not the original
people_mask = people[['weight','birthyear','children']]
people_mask

,weight,birthyear,children
state,,,
CA,68,1985,NaN
NY,83,1984,3.0
WY,112,1992,0.0


In [12]:
# Here a system view so we change the copy and we got settingwithCopyWarning
people_mask.loc['children'] = 999

C:\Users\ismail.berrada\.conda\envs\dna-2021-2022\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [13]:
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,68,1985,NaN,Biking,2000-01-01
NY,83,1984,3.0,Dancing,2000-01-02
WY,112,1992,0.0,NaN,2000-01-03


In [14]:
people_mask

,weight,birthyear,children
state,,,
CA,68,1985,NaN
NY,83,1984,3.0
WY,112,1992,0.0
children,999,999,999.0


In [15]:
#not sharing with people
people_mask.to_numpy().base is people.to_numpy().base

False

In [16]:
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,68,1985,NaN,Biking,2000-01-01
NY,83,1984,3.0,Dancing,2000-01-02
WY,112,1992,0.0,NaN,2000-01-03


In [17]:
# here it is a copy of the original
people_slice = people.loc['CA':'NY']
people_slice

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,68,1985,NaN,Biking,2000-01-01
NY,83,1984,3.0,Dancing,2000-01-02


In [18]:
#not sharing with people
people_slice['weight'] = 11.0

C:\Users\ISMAIL~1.BER\AppData\Local\Temp/ipykernel_70064/3862190093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  people_slice['weight'] = 11.0


In [19]:
people_slice

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,11.0,1985,NaN,Biking,2000-01-01
NY,11.0,1984,3.0,Dancing,2000-01-02


In [20]:
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,68,1985,NaN,Biking,2000-01-01
NY,83,1984,3.0,Dancing,2000-01-02
WY,112,1992,0.0,NaN,2000-01-03


### Flipping column content

In [84]:
# data are modified
people[['weight', 'birthyear']] = people[['birthyear', 'weight']]
people

,weight,birthyear,children,hobby
state,,,,
CA,1985,68,NaN,Biking
NY,1984,83,3.0,Dancing
WY,1992,112,0.0,NaN


#### **but here no modification because: pandas aligns all AXES when setting Series and DataFrame from .loc, and .iloc.**

In [70]:
pd.__version__

'1.1.1'

In [71]:
people.loc[:,['weight', 'birthyear']] = people[['birthyear', 'weight']]
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,1985,11,NaN,Biking,2000-01-01
NY,1984,11,3.0,Dancing,2000-01-02
WY,1992,112,0.0,NaN,2000-01-03


In [72]:
#use this to prevent alignment
people.loc[:,['weight', 'birthyear']] = people[['birthyear', 'weight']].values
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,11,1985,NaN,Biking,2000-01-01
NY,11,1984,3.0,Dancing,2000-01-02
WY,112,1992,0.0,NaN,2000-01-03


In [73]:
people[['weight', 'birthyear']] = people.loc[:,['birthyear', 'weight']]
people

,weight,birthyear,children,hobby,new_index
state,,,,,
CA,1985,11,NaN,Biking,2000-01-01
NY,1984,11,3.0,Dancing,2000-01-02
WY,1992,112,0.0,NaN,2000-01-03


## Adding and removing columns
You can generally treat `DataFrame` objects like dictionaries of `Series`, so the following work fine:

In [85]:
people

,weight,birthyear,children,hobby
state,,,,
CA,1985,68,NaN,Biking
NY,1984,83,3.0,Dancing
WY,1992,112,0.0,NaN


In [74]:
people["age"] = 2018 - people["birthyear"]  # adds a new column "age"
people["over 30"] = people["age"] > 30      # adds another column "over 30"
birthyears = people.pop("birthyear")
del people["children"]

people

,weight,hobby,new_index,age,over 30
state,,,,,
CA,1985,Biking,2000-01-01,2007,True
NY,1984,Dancing,2000-01-02,2007,True
WY,1992,NaN,2000-01-03,1906,True


In [72]:
birthyears

state
CA    1985
NY    1984
WY    1992
Name: birthyear, dtype: int64

When you add a new colum, it must have the same number of rows. Missing rows are filled with NaN, and extra rows are ignored:

In [86]:
people["pets"] = pd.Series({"bob": 0, "charles": 5, "eugene":1})  # alice is missing, eugene is ignored
people

,weight,birthyear,children,hobby,pets
state,,,,,
CA,1985,68,NaN,Biking,NaN
NY,1984,83,3.0,Dancing,NaN
WY,1992,112,0.0,NaN,NaN


When adding a new column, it is added at the end (on the right) by default. You can also insert a column anywhere else using the `insert()` method:

In [87]:
people.insert(1, "height", [172, 181, 185])
people

,weight,height,birthyear,children,hobby,pets
state,,,,,,
CA,1985,172,68,NaN,Biking,NaN
NY,1984,181,83,3.0,Dancing,NaN
WY,1992,185,112,0.0,NaN,NaN


In [88]:
people.drop('pets',axis=1)

,weight,height,birthyear,children,hobby
state,,,,,
CA,1985,172,68,NaN,Biking
NY,1984,181,83,3.0,Dancing
WY,1992,185,112,0.0,NaN


In [89]:
people

,weight,height,birthyear,children,hobby,pets
state,,,,,,
CA,1985,172,68,NaN,Biking,NaN
NY,1984,181,83,3.0,Dancing,NaN
WY,1992,185,112,0.0,NaN,NaN


In [90]:
people.drop('pets',axis=1,inplace=True)
people

,weight,height,birthyear,children,hobby
state,,,,,
CA,1985,172,68,NaN,Biking
NY,1984,181,83,3.0,Dancing
WY,1992,185,112,0.0,NaN


Can also drop rows this way:

In [91]:
people.drop('CA')

,weight,height,birthyear,children,hobby
state,,,,,
NY,1984,181,83,3.0,Dancing
WY,1992,185,112,0.0,NaN


## Assigning new columns
You can also create new columns by calling the `assign()` method. Note that this returns a new `DataFrame` object, the original is not modified:

In [95]:
people.assign(
    body_mass_index = people["weight"] / (people["height"] / 100) ** 2,
    has_pets = people["children"] > 0
)

,weight,height,birthyear,children,hobby,body_mass_index,has_pets
state,,,,,,,
CA,1985,172,68,NaN,Biking,670.970795,False
NY,1984,181,83,3.0,Dancing,605.598120,True
WY,1992,185,112,0.0,NaN,582.030679,False


Note that you cannot access columns created within the same assignment. You can pass a function to the `assign()` method (typically a `lambda` function), and this function will be called with the `DataFrame` as a parameter:

In [49]:
(people
     .assign(body_mass_index = lambda df: df["weight"] / (df["height"] / 100) ** 2)
     .assign(overweight = lambda df: df["body_mass_index"] > 25)
)

,birthyear,children,height,hobby,pets,weight,body_mass_index,overweight
bob,1984,3.0,181,Dancing,0.0,83,25.335002,True
alice,1985,NaN,172,Biking,NaN,68,22.985398,False
charles,1992,0.0,185,NaN,5.0,112,32.724617,True


Problem solved!

## Evaluating an expression
A great feature supported by pandas is expression evaluation. This relies on the `numexpr` library which must be installed.

In [50]:
people

,birthyear,children,height,hobby,pets,weight
bob,1984,3.0,181,Dancing,0.0,83
alice,1985,NaN,172,Biking,NaN,68
charles,1992,0.0,185,NaN,5.0,112


In [51]:
people.eval("B = weight / (height/100) ** 2 > 25",inplace=False)
people

,birthyear,children,height,hobby,pets,weight
bob,1984,3.0,181,Dancing,0.0,83
alice,1985,NaN,172,Biking,NaN,68
charles,1992,0.0,185,NaN,5.0,112


Assignment expressions are also supported. Let's set `inplace=True` to directly modify the `DataFrame` rather than getting a modified copy:

In [52]:
people.eval("body_mass_index = weight / (height/100) ** 2",inplace=True)
people

,birthyear,children,height,hobby,pets,weight,body_mass_index
bob,1984,3.0,181,Dancing,0.0,83,25.335002
alice,1985,NaN,172,Biking,NaN,68,22.985398
charles,1992,0.0,185,NaN,5.0,112,32.724617


## Sorting a `DataFrame`
You can sort a `DataFrame` by calling its `sort_index` method. By default it sorts the rows by their index label, in ascending order, but let's reverse the order:

In [96]:
people.sort_index(ascending=False)

,weight,height,birthyear,children,hobby
state,,,,,
WY,1992,185,112,0.0,NaN
NY,1984,181,83,3.0,Dancing
CA,1985,172,68,NaN,Biking


Note that `sort_index` returned a sorted *copy* of the `DataFrame`. To modify `people` directly, we can set the `inplace` argument to `True`. Also, we can sort the columns instead of the rows by setting `axis=1`:

In [97]:
people.sort_index(axis=1, inplace=True)
people

,birthyear,children,height,hobby,weight
state,,,,,
CA,68,NaN,172,Biking,1985
NY,83,3.0,181,Dancing,1984
WY,112,0.0,185,NaN,1992


To sort the `DataFrame` by the values instead of the labels, we can use `sort_values` and specify the column to sort by:

In [98]:
people.sort_values(by="weight", inplace=True)
people

,birthyear,children,height,hobby,weight
state,,,,,
NY,83,3.0,181,Dancing,1984
CA,68,NaN,172,Biking,1985
WY,112,0.0,185,NaN,1992


Again, there are way too many options to list here: the best option is to scroll through the [Visualization](http://pandas.pydata.org/pandas-docs/stable/visualization.html) page in pandas' documentation, find the plot you are interested in and look at the example code.

## Operations on `DataFrames`
Although `DataFrame`s do not try to mimick NumPy arrays, there are a few similarities. Let's create a `DataFrame` to demonstrate this:

In [100]:
grades_array = np.array([[8,8,9],[10,9,9],[4, 8, 2], [9, 10, 10]])
grades = pd.DataFrame(grades_array, columns=["sep", "oct", "nov"], index=["alice","bob","charles","darwin"])
grades

,sep,oct,nov
alice,8,8,9
bob,10,9,9
charles,4,8,2
darwin,9,10,10


You can apply NumPy mathematical functions on a `DataFrame`: the function is applied to all values:

In [101]:
np.sqrt(grades)

,sep,oct,nov
alice,2.828427,2.828427,3.000000
bob,3.162278,3.000000,3.000000
charles,2.000000,2.828427,1.414214
darwin,3.000000,3.162278,3.162278


Similarly, adding a single value to a `DataFrame` will add that value to all elements in the `DataFrame`. This is called *broadcasting*:

In [102]:
grades + 1

,sep,oct,nov
alice,9,9,10
bob,11,10,10
charles,5,9,3
darwin,10,11,11


Of course, the same is true for all other binary operations, including arithmetic (`*`,`/`,`**`...) and conditional (`>`, `==`...) operations:

In [103]:
grades >= 5

,sep,oct,nov
alice,True,True,True
bob,True,True,True
charles,False,True,False
darwin,True,True,True


Aggregation operations, such as computing the `max`, the `sum` or the `mean` of a `DataFrame`, apply to each column, and you get back a `Series` object:

In [105]:
grades.mean(axis=1)

alice      8.333333
bob        9.333333
charles    4.666667
darwin     9.666667
dtype: float64

The `all` method is also an aggregation operation: it checks whether all values are `True` or not. Let's see during which months all students got a grade greater than `5`:

In [96]:
(grades > 5).all()

sep    False
oct     True
nov    False
dtype: bool

Most of these functions take an optional `axis` parameter which lets you specify along which axis of the `DataFrame` you want the operation executed. The default is `axis=0`, meaning that the operation is executed vertically (on each column). You can set `axis=1` to execute the operation horizontally (on each row). For example, let's find out which students had all grades greater than `5`:

In [97]:
(grades > 5).all(axis = 1)

alice       True
bob         True
charles    False
darwin      True
dtype: bool

The `any` method returns `True` if any value is True. Let's see who got at least one grade 10:

In [98]:
(grades == 10).any(axis = 1)

alice      False
bob         True
charles    False
darwin      True
dtype: bool

If you add a `Series` object to a `DataFrame` (or execute any other binary operation), pandas attempts to broadcast the operation to all *rows* in the `DataFrame`. This only works if the `Series` has the same size as the `DataFrame`s rows. For example, let's substract the `mean` of the `DataFrame` (a `Series` object) from the `DataFrame`:

In [99]:
grades - grades.mean()  # equivalent to: grades - [7.75, 8.75, 7.50]

,sep,oct,nov
alice,0.25,-0.75,1.5
bob,2.25,0.25,1.5
charles,-3.75,-0.75,-5.5
darwin,1.25,1.25,2.5


We substracted `7.75` from all September grades, `8.75` from October grades and `7.50` from November grades. It is equivalent to substracting this `DataFrame`:

In [100]:
pd.DataFrame([[7.75, 8.75, 7.50]]*4, index=grades.index, columns=grades.columns)

,sep,oct,nov
alice,7.75,8.75,7.5
bob,7.75,8.75,7.5
charles,7.75,8.75,7.5
darwin,7.75,8.75,7.5


If you want to substract the global mean from every grade, here is one way to do it:

In [101]:
grades - grades.values.mean() # substracts the global mean (8.00) from all grades

,sep,oct,nov
alice,0.0,0.0,1.0
bob,2.0,1.0,1.0
charles,-4.0,0.0,-6.0
darwin,1.0,2.0,2.0


## Automatic alignment
Similar to `Series`, when operating on multiple `DataFrame`s, pandas automatically aligns them by row index label, but also by column names. Let's create a `DataFrame` with bonus points for each person from October to December:

In [106]:
grades

,sep,oct,nov
alice,8,8,9
bob,10,9,9
charles,4,8,2
darwin,9,10,10


In [107]:
bonus_array = np.array([[0,np.nan,2],[np.nan,1,0],[0, 1, 0], [3, 3, 0]])
bonus_array

array([[ 0., nan,  2.],
       [nan,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 3.,  3.,  0.]])

In [108]:
bonus_points = pd.DataFrame(bonus_array, columns=["oct", "nov", "dec"], index=["bob","colin", "darwin", "charles"])
bonus_points

,oct,nov,dec
bob,0.0,NaN,2.0
colin,NaN,1.0,0.0
darwin,0.0,1.0,0.0
charles,3.0,3.0,0.0


In [69]:
grades

,sep,oct,nov
alice,8,8,9
bob,10,9,9
charles,4,8,2
darwin,9,10,10


In [109]:
grades + bonus_points

,dec,nov,oct,sep
alice,NaN,NaN,NaN,NaN
bob,NaN,NaN,9.0,NaN
charles,NaN,5.0,11.0,NaN
colin,NaN,NaN,NaN,NaN
darwin,NaN,11.0,10.0,NaN


- For more details: https://stackoverflow.com/questions/51645195/pandas-align-function-illustrative-example/51645550

## Multi-Index and Index Hierarchy

Let us go over how to work with Multi-Index, first we'll create a quick example of what a Multi-Indexed DataFrame would look like:

In [241]:
import pandas as pd
import numpy as np
# Index Levels
outside = ['G1','G1','G1','G2','G2','G2']
inside = [1,2,3,1,2,3]
hier_index = list(zip(outside,inside))
hier_index = pd.MultiIndex.from_tuples(hier_index)

In [242]:
hier_index

MultiIndex([('G1', 1),
            ('G1', 2),
            ('G1', 3),
            ('G2', 1),
            ('G2', 2),
            ('G2', 3)],
           )

In [243]:
df = pd.DataFrame(np.random.randn(6,2),index=hier_index,columns=['A','B'])
df

A         B
G1 1  0.814470 -1.170740
   2 -0.384270 -1.018994
   3 -0.704999  0.139643
G2 1 -1.079552 -0.073198
   2 -1.511438  0.379933
   3  2.183257 -1.081558

Now let's show how to index this! For index hierarchy we use df.loc[], if this was on the columns axis, you would just use normal bracket notation df[]. Calling one level of the index returns the sub-dataframe:

In [244]:
df.loc['G1']

,A,B
1,0.814470,-1.170740
2,-0.384270,-1.018994
3,-0.704999,0.139643


In [245]:
df.loc['G1'].loc[1]

A    0.81447
B   -1.17074
Name: 1, dtype: float64

In [246]:
df.index.names

FrozenList([None, None])

In [247]:
df.index.names = ['Group','Num']

In [248]:
df

A         B
Group Num                    
G1    1    0.814470 -1.170740
      2   -0.384270 -1.018994
      3   -0.704999  0.139643
G2    1   -1.079552 -0.073198
      2   -1.511438  0.379933
      3    2.183257 -1.081558

In [249]:
df.xs('G1')

,A,B
Num,,
1,0.814470,-1.170740
2,-0.384270,-1.018994
3,-0.704999,0.139643


In [250]:
df.xs(['G1',1])

A    0.81447
B   -1.17074
Name: (G1, 1), dtype: float64